In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.display import clear_output
from tkinter import Tk, filedialog
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import quick_pp.las_handler as las

In [ ]:
# Loading a dataframe

root = Tk()
file_objects = filedialog.askopenfiles(title='Choose well Log ASCII Standard (LAS) files to be combined',
                                        filetype=(('LAS Files', '*.LAS *.las'), ('All Files', '*.*')),
                                        multiple=True,
                                        mode='rb')
root.destroy()
df, _ = las.read_las_files(file_objects)

In [ ]:
print([c for c in df.columns if 'RT' in c])

In [ ]:
df.WELL_NAME.unique()

In [ ]:
from quick_pp.core_calibration import fit_poroperm_curve, poroperm_xplot

poroperm_constants = {}
for rt, rt_data in df.groupby('RT_V2_2023'):
    a, b = fit_poroperm_curve(rt_data['CPORE'], rt_data['CPERM'])
    poroperm_constants[rt] = {'a': a, 'b': b}
    poroperm_xplot(rt_data['CPORE'], rt_data['CPERM'], a, b, label=f'RT {rt} - a: {a}, b: {b}')
    # plt.show()


In [ ]:
poroperm_constants

In [ ]:
from quick_pp.core_calibration import fit_poroperm_curve, poroperm_xplot

poroperm_constants = {
    1.0: {'a': 624556, 'b': 4.325},
    2.0: {'a': 100029, 'b': 3.832},
    3.0: {'a': 131015, 'b': 4.666},
    4.0: {'a': 26971, 'b': 4.308}
}
for rt, rt_data in df.groupby('RT_V2_2023'):
    a = poroperm_constants[rt]['a']
    b = poroperm_constants[rt]['b']    
    poroperm_xplot(rt_data['CPORE'], rt_data['CPERM'], a, b, label=f'RT {rt} - a: {a}, b: {b}')
    # plt.show()


In [ ]:
from quick_pp.core_calibration import fit_j_curve, j_xplot

well_name = 'BA-H010'
well_df = df[df['WELL_NAME'] == well_name]
well_df.dropna(axis=1, how='all', inplace=True)

j_constants = {}
for rt, well_data in well_df.groupby('RT_V2_2023'):
    temp_df = pd.DataFrame()
    for i, data in well_data.iterrows():
        t_df = data[[c for c in well_data.columns if 'SW[' in c]].reset_index(drop=True).T.reset_index(name='SW')
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'PC' in c]].reset_index(drop=True).T.reset_index(name='PC')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'SWN' in c]].reset_index(drop=True).T.reset_index(name='SWN')], axis=1)
        t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'J' in c]].reset_index(drop=True).T.reset_index(name='J')], axis=1)
        t_df.drop(columns='index', inplace=True)
        temp_df = pd.concat([temp_df, t_df])
    temp_df.dropna(inplace=True)
    temp_df = temp_df[(temp_df > 0) & (temp_df != np.inf)]
    temp_df = temp_df.sort_values(by='SW').reset_index(drop=True)
    a, b = fit_j_curve(temp_df['SW'], temp_df['J'])
    j_constants[rt] = {'a': a, 'b': b}
    j_xplot(temp_df['SW'], temp_df['J'], a, b, label=f"RT {rt} - a: {a}, b: {b}")
    # plt.show()


In [ ]:
j_constants

In [ ]:
from quick_pp.core_calibration import fit_j_curve, j_xplot

well_name = 'BA-H010'
well_df = df[df['WELL_NAME'] == well_name]
well_df.dropna(axis=1, how='all', inplace=True)

j_constants = {
    # 3.0: {'a': 0.507, 'b': -2.445}, 
    # 4.0: {'a': 0.583, 'b': -1.969}
    3.0: {'a': 0.0271, 'b': -2.653}, 
    4.0: {'a': 0.0363, 'b': -3.483}
}
for rt, well_data in well_df.groupby('RT_V2_2023'):
    temp_df = pd.DataFrame()
    for i, data in well_data.iterrows():
        t_df = data[[c for c in well_data.columns if 'SW[' in c]].reset_index(drop=True).T.reset_index(name='SW')
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'PC' in c]].reset_index(drop=True).T.reset_index(name='PC')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'SWN' in c]].reset_index(drop=True).T.reset_index(name='SWN')], axis=1)
        t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'J' in c]].reset_index(drop=True).T.reset_index(name='J')], axis=1)
        t_df.drop(columns='index', inplace=True)
        temp_df = pd.concat([temp_df, t_df])
    temp_df.dropna(inplace=True)
    temp_df = temp_df[(temp_df > 0) & (temp_df != np.inf)]
    temp_df = temp_df.sort_values(by='SW').reset_index(drop=True)
    a = j_constants[rt]['a']
    b = j_constants[rt]['b']
    j_xplot(temp_df['SW'], temp_df['J'], a, b, label=f"RT {rt} - a: {a}, b: {b}")
    # plt.show()


In [ ]:
# Merge core data with well data
well_df_core = well_df.dropna(subset=['CPORE', 'CPERM']).sort_values(by='DEPTH')
filtered_df = pd.merge_asof(barm_h010, well_df_core[['DEPTH', 'CPERM', 'CPORE']], on='DEPTH', direction='nearest', tolerance=0.5)

In [ ]:
# filtered_df.to_csv('barm_h010_core.csv', index=False)
filtered_df = pd.read_csv('barm_h010_core.csv')

In [ ]:
min_depth = 4650
max_depth = 4745
test = filtered_df[(filtered_df.DEPTH > min_depth) & (filtered_df.DEPTH < max_depth)][['DEPTH', 'PHIT', 'CPORE']]
plt.figure(figsize=(20, 3))
plt.plot(test.DEPTH, test.PHIT, label='PHIT')
plt.scatter(x=test.DEPTH, y=test.CPORE, label='CPORE')

In [ ]:
min_depth = 4650
max_depth = 4745
test = filtered_df[(filtered_df.DEPTH > min_depth) & (filtered_df.DEPTH < max_depth)][['DEPTH', 'PERM', 'CPERM']]
plt.figure(figsize=(20, 3))
plt.plot(test.DEPTH, test.PERM, label='PERM')
plt.scatter(x=test.DEPTH, y=test.CPERM, label='CPERM')
plt.yscale('log')

In [ ]:
from quick_pp.core_calibration import sw_shf_leverett_j
min_depth = 4600
max_depth = 4745
test = filtered_df[(filtered_df.DEPTH > min_depth) & (filtered_df.DEPTH < max_depth)].copy()
# TODO: groupby RT and use
test['SW_SHF'] = sw_shf_leverett_j(
    test['PERM'], test['PHIT'], test['DEPTH'], 
    fwl=4680,
    ift=26,
    gw=0.433,
    ghc=0.35,
    a=.038271,
    b=-5.33
)
plt.figure(figsize=(20, 3))
plt.plot(test.DEPTH, test.SWT, label='SWT')
plt.plot(test.DEPTH, test.SW_SHF, label='SW_SHF')
plt.legend()

In [ ]:
from quick_pp.rock_type import rock_typing, rqi, fzi, vsh_gr
filtered_df['RQI'] = rqi(filtered_df['PERM'], filtered_df['PHIT'])
filtered_df['FZI'] = fzi(filtered_df['PERM'], filtered_df['PHIT'])
filtered_df['VSH_GR'] = vsh_gr(filtered_df['GR'])
filtered_df['ROCK_FLAG'] = rock_typing(filtered_df['FZI'], [2.3, 4.1, 6.0], False)
# filtered_df['ROCK_FLAG'] = rock_typing(filtered_df['VSH_GR'])


for rt, data in filtered_df.groupby('ROCK_FLAG'):
    a, b = fit_poroperm_curve(data['PHIT'], data['PERM'])
    poroperm_xplot(data['PHIT'], data['PERM'], a, b, label=f'RT {rt} - a: {a}, b: {b}')


In [ ]:
from quick_pp.core_calibration import fit_j_curve, bvw_xplot

well_name = 'BA-H010'
well_df = df[df['WELL_NAME'] == well_name]
well_df.dropna(axis=1, how='all', inplace=True)

for rt, well_data in well_df.groupby('RT_V2_2023'):
    temp_df = pd.DataFrame()
    for i, data in well_data.iterrows():
        cpore = data['CPORE']
        t_df = data[[c for c in well_data.columns if 'SW[' in c]].reset_index(drop=True).T.reset_index(name='SW')
        t_df['BVW'] = t_df['SW'] * cpore
        t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'PC' in c]].reset_index(drop=True).T.reset_index(name='PC')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'SWN' in c]].reset_index(drop=True).T.reset_index(name='SWN')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in well_data.columns if 'J' in c]].reset_index(drop=True).T.reset_index(name='J')], axis=1)
        t_df.drop(columns='index', inplace=True)
        temp_df = pd.concat([temp_df, t_df])
    temp_df.dropna(inplace=True)
    temp_df = temp_df[(temp_df > 0) & (temp_df != np.inf)]
    temp_df = temp_df.sort_values(by='SW').reset_index(drop=True)
    a, b = fit_j_curve(temp_df['BVW'], temp_df['PC'])
    bvw_xplot(temp_df['BVW'], temp_df['PC'], a, b, label=f"RT {rt} - a: {a}, b: {b}")
    # plt.show()


In [ ]:
from quick_pp.core_calibration import sw_shf_cuddy
min_depth = 4600
max_depth = 4745
test = filtered_df[(filtered_df.DEPTH > min_depth) & (filtered_df.DEPTH < max_depth)].copy()

test['SHF'] = sw_shf_cuddy(test['PHIT'], test['DEPTH'],
                            fwl=4750, 
                            gw=0.433, ghc=0.35, a=0.27, b=-2.0124)

plt.figure(figsize=(20, 3))
plt.plot(test.DEPTH, test.SWT, label='SWT')
plt.plot(test.DEPTH, test.SHF, label='SHF')
plt.legend()

In [ ]:
from quick_pp.core_calibration import sw_shf_cuddy
filtered_df['SHF'] = sw_shf_cuddy(filtered_df['PHIT'], filtered_df['DEPTH'], fwl=4750, gw=0.433, ghc=0.35, a=0.27, b=-2.0124)

In [ ]:
from quick_pp.core_calibration import sw_shf_leverett_j

j_constants = {
    1: {'a': 0.0271, 'b': -2.653},
    2: {'a': 0.0363, 'b': -3.483},
    3: {'a': 0.0382, 'b': -5.330}, 
    4: {'a': 0.0563, 'b': -8.483}
}
for rt, well_data in filtered_df.groupby('ROCK_FLAG'):
    a = j_constants[rt]['a']
    b = j_constants[rt]['b']
    filtered_df.loc[filtered_df['ROCK_FLAG'] == rt, 'SHF'] = sw_shf_leverett_j(
        well_data['PERM'], well_data['PHIT'], well_data['DEPTH'], 
        fwl=4680,
        ift=26,
        gw=0.433,
        ghc=0.35,
        a=a,
        b=b
    )

In [ ]:
from quick_pp.plotter import plotly_log

plotly_log(filtered_df)

In [ ]:
from quick_pp.core_calibration import estimate_hafwl

hafwl = estimate_hafwl(
    sw=well_df['SW[0]'],
    poro=well_df['CPORE'],
    perm=well_df['CPERM'],
    ift=26,
    gw=1.0,
    ghc=0.7,
    a=0.507,
    b=2.455
)
hafwl

In [ ]:
# # Need HAFWL to plot 
# df.plot(x='BVW', y='DEPTH_SHIFTED', kind='scatter', logy=False, xlim=(0, 0.5))

In [ ]:
# df[[c for c in df.columns if 'PC' in c]].describe()

In [ ]:
# for i in range(8):
#     df.plot(x=f'SW[{i}]', y=f'PC[{i}]', kind='scatter', logy=False, xlim=(0, 1))
# plt.show()